# Предсказание выходной температуры сплава при плавке стали 

Чтобы оптимизировать производственные расходы, металлургический комбинат ООО «Так закаляем сталь» решил уменьшить потребление электроэнергии на этапе обработки стали. Кратко, процесс плавки представляет собой замер температуры, нагревание сплава электродами, добавление примесей, продувка газом и снова замер температуры. Процесс повторяется необходимое количество раз.  

**Цель работы:**  
Создать модель прогнозирования температуры сплава на последнем цикле плавки. Исходные данные - техническая информация(циклы нагрева и мощность электродов, информация о примесях, продувке газом и пр.) для чуть более 3000 процессов плавки.  
Метрика качества - MAE. Она должна быть не выше 6.8.

**Ход работы**
1. Исследование данных
2. Предобработка данных
3. Выбор и исследование нескольких моделей машинного обучения
4. Тестирование наиболее подходящей модели
5. Создание отчета по итогам проведенной работы

# Исследование данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Импортируем наши датасеты

In [ ]:
try:
    data_arc = pd.read_csv('data_arc_new.csv')
    data_bulk_time = pd.read_csv('data_bulk_time_new.csv')
    data_gas = pd.read_csv('data_gas_new.csv')
    data_temp = pd.read_csv('data_temp_new.csv')
    data_temp = pd.read_csv('data_temp_new.csv')
    data_wire_time = pd.read_csv('data_wire_time_new.csv')

except:
    data_arc = pd.read_csv('/datasets/data_arc_new.csv')
    data_bulk_time = pd.read_csv('/datasets/data_bulk_time_new.csv')
    data_gas = pd.read_csv('/datasets/data_gas_new.csv')
    data_temp = pd.read_csv('/datasets/data_temp_new.csv')
    data_temp = pd.read_csv('/datasets/data_temp_new.csv')
    data_wire_time = pd.read_csv('/datasets/data_wire_time_new.csv')

Так как работаем мы с несколькими датасетами сразу, исследуем их по очереди, чтобы не запутаться

## Data_arc

`data_arc` - это датасет с данными об электродах, которые опускают в ковш, для плавки находящегося в нем стального сплава